## Use word embedding layers for deep learning with Keras

Word embedding provide dense representation and relative meanings.

In [1]:
#define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [2]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [3]:
#integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [4]:
print(encoded_docs)

[[45, 28], [34, 26], [40, 19], [38, 26], [10], [9], [8, 19], [49, 34], [8, 26], [6, 11, 28, 18]]


In [5]:
#pad the documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


In [6]:
print(padded_docs)

[[45 28  0  0]
 [34 26  0  0]
 [40 19  0  0]
 [38 26  0  0]
 [10  0  0  0]
 [ 9  0  0  0]
 [ 8 19  0  0]
 [49 34  0  0]
 [ 8 26  0  0]
 [ 6 11 28 18]]


In [7]:
#define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [8]:
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [9]:
#summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
#fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)

Epoch 1/50
10/10 [==============================] - 0s - loss: 0.6919 - acc: 0.5000
Epoch 2/50
10/10 [==============================] - 0s - loss: 0.6907 - acc: 0.5000
Epoch 3/50
10/10 [==============================] - 0s - loss: 0.6896 - acc: 0.5000
Epoch 4/50
10/10 [==============================] - 0s - loss: 0.6884 - acc: 0.5000
Epoch 5/50
10/10 [==============================] - 0s - loss: 0.6873 - acc: 0.6000
Epoch 6/50
10/10 [==============================] - 0s - loss: 0.6861 - acc: 0.6000
Epoch 7/50
10/10 [==============================] - 0s - loss: 0.6849 - acc: 0.6000
Epoch 8/50
10/10 [==============================] - 0s - loss: 0.6838 - acc: 0.6000
Epoch 9/50
10/10 [==============================] - 0s - loss: 0.6826 - acc: 0.6000
Epoch 10/50
10/10 [==============================] - 0s - loss: 0.6814 - acc: 0.6000
Epoch 11/50
10/10 [==============================] - 0s - loss: 0.6802 - acc: 0.6000
Epoch 12/50
10/10 [==============================] - 0s - loss: 0.6791 - a

In [11]:
#evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)

10/10 [==============================] - 0s


In [12]:
print("Accuracy : %f"%(accuracy*100))

Accuracy : 89.999998


## Using Pre trained GloVe Embedding

In [14]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [15]:
#define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [16]:
#prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index)+1

In [17]:
#integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [18]:
#pad documents to max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [32]:
#load the whole embedding into memory
embeddings_index = dict()

f = open('../../../myWork/kaggle/DATASET/glove.6B/glove.6B.100d.txt')

In [33]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [34]:
print('Loaded %s word vectors.'%len(embeddings_index))

Loaded 400000 word vectors.


In [35]:
#create a weight matrix for words in training docs
embeddings_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [36]:
#define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embeddings_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [37]:
#compile the model
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["acc"])

In [38]:
#summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None


In [39]:
#fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)

Epoch 1/50
10/10 [==============================] - 0s - loss: 0.7244 - acc: 0.5000
Epoch 2/50
10/10 [==============================] - 0s - loss: 0.7057 - acc: 0.6000
Epoch 3/50
10/10 [==============================] - 0s - loss: 0.6878 - acc: 0.6000
Epoch 4/50
10/10 [==============================] - 0s - loss: 0.6707 - acc: 0.6000
Epoch 5/50
10/10 [==============================] - 0s - loss: 0.6545 - acc: 0.6000
Epoch 6/50
10/10 [==============================] - 0s - loss: 0.6391 - acc: 0.6000
Epoch 7/50
10/10 [==============================] - 0s - loss: 0.6246 - acc: 0.6000
Epoch 8/50
10/10 [==============================] - 0s - loss: 0.6109 - acc: 0.8000
Epoch 9/50
10/10 [==============================] - 0s - loss: 0.5980 - acc: 0.8000
Epoch 10/50
10/10 [==============================] - 0s - loss: 0.5858 - acc: 0.8000
Epoch 11/50
10/10 [==============================] - 0s - loss: 0.5745 - acc: 0.8000
Epoch 12/50
10/10 [==============================] - 0s - loss: 0.5638 - a

In [40]:
#evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print("Accuracy : %f"%(accuracy*100))

10/10 [==============================] - 0s
Accuracy : 100.000000
